# Build a Simple LLM application with chat models and prompt templates

In [7]:
pip install -q langchain python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
pip freeze

aiofiles==24.1.0
aiohappyeyeballs==2.4.3
aiohttp==3.10.10
aiosignal==1.3.1
annotated-types==0.7.0
anyio==4.6.2
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
async-timeout==4.0.3
attrs==24.2.0
babel==2.17.0
beautifulsoup4==4.12.3
bleach==6.2.0
blinker==1.9.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.4.0
click==8.1.7
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.14
decorator==5.2.1
defusedxml==0.7.1
Deprecated==1.2.14
dirtyjson==1.0.8
distlib==0.3.9
distro==1.9.0
exceptiongroup==1.2.2
executing==2.2.0
fastapi==0.115.12
fastjsonschema==2.21.1
ffmpy==0.5.0
filelock==3.16.1
Flask==3.1.0
flask-mysql-connector==1.1.0
fqdn==1.5.1
frozenlist==1.4.1
fsspec==2024.9.0
gradio==5.27.0
gradio_client==1.9.0
greenlet==3.1.1
groovy==0.1.2
groq==0.23.1
h11==0.14.0
httpcore==1.0.6
httpx==0.27.2
httpx-sse==0.4.0
huggingface-hub==0.30.2
idna==3.10
iniconfig==2.0.0
ipykernel==6.29.5
ipython==8.36.0
ipywidgets==8.1.6
isoduration=

In [7]:
pip install -q langchain python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Create a `.env` file using Python
with open('.env', 'w') as f:
    f.write("MY_API_KEY=your_api_key_here\n")
    f.write("SECRET_KEY=your_secret_key_here\n")


### LangSmith

In [1]:
import getpass
import os

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass
    
os.environ["LANGSMIT_TRACING"] = 'true'
if "LANGSMIT_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt = "Enter your Langsmith API key(optional):"
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt = 'Enter your LangSmith API Key(Optional): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = 'default'

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass.getpass(
        prompt = 'Enter your GROQ API key (required if using GROQ)'
    )




Enter your Langsmith API key(optional): ········
Enter your LangSmith API Key(Optional):  ········
Enter your GROQ API key (required if using GROQ) ········


### Using Language Models

In [50]:
import getpass
import os

if not os.environ.get('GROQ_API_KEY'):
    os.environ['GROQ_API_KEY'] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model
model = init_chat_model('llama3-8b-8192', model_provider = 'groq')


In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English to Hindi"),
    HumanMessage("Hi!"),
]
model.invoke(messages)

AIMessage(content='नमस्ते! (Namaste!)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 24, 'total_tokens': 35, 'completion_time': 0.009166667, 'prompt_time': 0.005659606, 'queue_time': 0.256383964, 'total_time': 0.014826273}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-efa91fc6-b387-4985-9e33-a843658697ae-0', usage_metadata={'input_tokens': 24, 'output_tokens': 11, 'total_tokens': 35})

In [4]:
model.invoke("Hello")
model.invoke([{
    'role': 'assistant',
    'content': 'Hello'
}])
model.invoke([HumanMessage("Hello")])

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.020833333, 'prompt_time': 0.001812125, 'queue_time': 0.236654546, 'total_time': 0.022645458}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-8cbd7249-ebd0-4ca9-a6e3-be061ec9a932-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

### Streaming

In [5]:
for token in model.stream(messages):
    print(token.content, end= '|')

|न|म|स|्त|े|!| (|Nam|aste|!)||

### Prompt Templates

In [45]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [('system', system_template), ('user', '{text}')]
)

prompt = prompt_template.invoke({"language": "Hindi", "text": input("Enter text:")
    }
)
prompt

Enter text: tell me about yourself


ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='tell me about yourself', additional_kwargs={}, response_metadata={})])

In [47]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='tell me about yourself', additional_kwargs={}, response_metadata={})]

### Invoking the Chat model on the formatted prompt


In [48]:
response = model.invoke(prompt)
print(response.content)

Here's the translation:

मैं कौन हूँ?

(Maine koun hoon?)

or

मेरे बारे में बताएं

(Mere bare mein bataen)

Both of these translations roughly mean "Tell me about yourself".
